In [1]:
#from tensorflow_docs.vis import embed
from tensorflow import keras
#from imutils import paths

#import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
#import imageio
import cv2
import os
import json

In [2]:
#Prevent tensorflow to allocate the entire GPU
#https://www.tensorflow.org/api_docs/python/tf/config/experimental/set_memory_growth
physical_devices = tf.config.list_physical_devices("GPU")
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    pass

2022-11-15 01:37:50.836246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 01:37:50.845618: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 01:37:50.846661: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [3]:
# train_df= pd.read_csv('/home/app/src/data/CSV/dataset_train_v2+v3.csv', )
# test_df= pd.read_csv('/home/app/src/data/CSV/dataset_test_v3.csv', )

In [3]:
train_df= pd.read_csv('/home/app/src/data/CSV/dataset_train_v2+v3_with_rep.csv')
test_df= pd.read_csv('/home/app/src/data/CSV/dataset_test_v2+v3_with_rep.csv')

In [4]:
train_df= train_df
test_df = test_df

print(f"Total videos for training: {len(train_df)}")
print(f"Total videos for testing: {len(test_df)}")

train_df.sample(10)

Total videos for training: 29586
Total videos for testing: 8149


,video_name,tag
6343,/home/app/src/data/shot-type-dataset/trailer_v...,Push
16034,/home/app/src/data/shot-type-dataset/trailer_v...,Motion
24850,/home/app/src/data/shot-type-dataset/trailer_v...,Static
9729,/home/app/src/data/shot-type-dataset/trailer_v...,Static
27746,/home/app/src/data/shot-type-dataset/trailer_v...,Push
21390,/home/app/src/data/shot-type-dataset/trailer_v...,Static
5606,/home/app/src/data/shot-type-dataset/trailer_v...,Motion
13422,/home/app/src/data/shot-type-dataset/trailer_v...,Static
4278,/home/app/src/data/shot-type-dataset/trailer_v...,Static
24581,/home/app/src/data/shot-type-dataset/trailer_v...,Push


Checking correct video tagging with original JSON

In [6]:
# # reading the JSON data using json.load()
# json_path = '/home/app/src/data/shot-type-dataset/v1_split_trailer.json'
# json_v2_path = '/home/app/src/data/shot-type-dataset/v2_full_trailer.json'

# with open(json_path) as train_file:
#     dict_v1= json.load(train_file)
# with open(json_v2_path) as v2_file:
#     dict_v2= json.load(v2_file)


# def tag_ckecker (df, json_dict1, json_dict2, df_name_to_check):
#     tag_chek_list = []

#     for i in range(len(df.index)):
#         movie = df['video_name'][i]
#         df_tag = df['tag'][df['video_name'] == movie][i]
#         find_char_st = movie.find('_', 55,60)
#         find_char_end = movie.find('_', 65,70)
#         movie_key = movie[find_char_st+1:find_char_end]
#         trailer_key =  movie[find_char_end+1:find_char_end+5]
#         try:
#             dict_tag = json_dict1[df_name_to_check][movie_key][trailer_key]['movement']['label']
#         except KeyError:
#             try:
#                 dict_tag = json_dict1['val'][movie_key][trailer_key]['movement']['label']
#                 if dict_tag == df_tag:
#                     pass
#                 else:
#                     tag_chek_list.append(movie)
#             except KeyError:
#                 dict_tag = json_dict2[movie_key][trailer_key]['movement']['label']
#                 if dict_tag == df_tag:
#                     pass
#                 else:
#                     tag_chek_list.append(movie)

            

#     if len(tag_chek_list) == 0:
#         print(f'tag check {df_name_to_check} OK')
#     else: print (tag_chek_list)


# tag_ckecker(train_df, dict_v1, dict_v2, 'train')
# tag_ckecker(test_df, dict_v1, dict_v2, 'test')


In [5]:
print(f'Train value counts:' '\n',train_df['tag'].value_counts(), '\n')
print(f'Test value counts:' '\n',test_df['tag'].value_counts(), '\n')


Train value counts:
 Push      530
Pull       76
Static      2
Motion      2
Name: tag, dtype: int64 

Test value counts:
 Static            5464
Motion            2385
Push               220
Pull                80
Multi_movement      38
Name: tag, dtype: int64 



Define hyperparameters

In [5]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 40

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

Loading frames and working on frame's size

In [7]:
def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

Feature extractor

In [8]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()
print(feature_extractor.summary())

2022-11-15 00:04:21.453274: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 00:04:21.453837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 00:04:21.454884: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 00:04:21.455855: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

87924736/87910968 [==============================] - 2s 0us/step
Model: "feature_extractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 tf.math.truediv (TFOpLambda  (None, 224, 224, 3)      0         
 )                                                               
                                                                 
 tf.math.subtract (TFOpLambd  (None, 224, 224, 3)      0         
 a)                                                              
                                                                 
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
Total params: 21,802,784
Trainable params: 21,768,352
Non-trainable params: 34,432
_________________________________

Working on labels

In [6]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train_df["tag"])
)
print(label_processor.get_vocabulary())

['Motion', 'Pull', 'Push', 'Static']


2022-11-15 01:38:20.787486: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-15 01:38:20.788115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 01:38:20.789110: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-15 01:38:20.789917: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

Preparing videos

In [11]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["tag"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()
        print(f' {root_dir}_video {idx}/{len(video_paths)} done')

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train")
print(f"Frame features in train set: {train_data[0].shape}")


2022-11-15 00:05:39.055568: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8200
2022-11-15 00:05:39.329045: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-15 00:05:39.329756: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-15 00:05:39.329810: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-11-15 00:05:39.330398: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-11-15 00:05:39.330505: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


 train_video 0/610 done
 train_video 1/610 done
 train_video 2/610 done
 train_video 3/610 done
 train_video 4/610 done
 train_video 5/610 done
 train_video 6/610 done
 train_video 7/610 done
 train_video 8/610 done
 train_video 9/610 done
 train_video 10/610 done
 train_video 11/610 done
 train_video 12/610 done
 train_video 13/610 done
 train_video 14/610 done
 train_video 15/610 done
 train_video 16/610 done
 train_video 17/610 done
 train_video 18/610 done
 train_video 19/610 done
 train_video 20/610 done
 train_video 21/610 done
 train_video 22/610 done
 train_video 23/610 done
 train_video 24/610 done
 train_video 25/610 done
 train_video 26/610 done
 train_video 27/610 done
 train_video 28/610 done
 train_video 29/610 done
 train_video 30/610 done
 train_video 31/610 done
 train_video 32/610 done
 train_video 33/610 done
 train_video 34/610 done
 train_video 35/610 done
 train_video 36/610 done
 train_video 37/610 done
 train_video 38/610 done
 train_video 39/610 done
 train_vid

Saving TRAIN embeddings

In [12]:
path_save_embeddings= '/home/app/src/embeddings/mov_v2_410_movies/'

# Train
train_data_embedding_0 = np.save(path_save_embeddings + 'train_data_embedding_0.npy', train_data[0])
train_data_embedding_1 = np.save(path_save_embeddings + 'train_data_embedding_1.npy', train_data[1])
train_labels_embedding = np.save(path_save_embeddings + 'train_labels_embedding.npy', train_labels)

: 

Preparing TEST videos

In [14]:
# test_data, test_labels = prepare_all_videos(test_df, "test")
# print(f"Frame masks in test set: {test_data[1].shape}")

Saving TEST embeddings

In [15]:
# #Test
# test_data_embedding_0 = np.save(path_save_embeddings + 'test_data_embedding_0.npy', test_data[0])
# test_data_embedding_1 = np.save(path_save_embeddings + 'test_data_embedding_1.npy', test_data[1])
# test_labels_embedding = np.save(path_save_embeddings + 'test_labels_embedding.npy', test_labels)

Loading embeddings

In [16]:
# path_load_embeddings= '/home/app/src/embeddings/mov_20856_train_8187_test_emb/20856_train_8187_test_10_frames/'

# #Train
# train_data_0 = np.load(path_load_embeddings + 'train_data_embedding_0_complete_10_frames.npy')
# train_data_1 = np.load(path_load_embeddings + 'train_data_embedding_1_complete_10_frames.npy')
# train_data=(train_data_0, train_data_1)
# #train_labels = np.load(path_load_embeddings + 'train_labels_embedding.npy')

# #Test
# test_data_0 = np.load(path_load_embeddings + 'test_data_embedding_0_complete_10_frames.npy')
# test_data_1 = np.load(path_load_embeddings + 'test_data_embedding_1_complete_10_frames.npy')
# test_data=(test_data_0, test_data_1)
# #test_labels = np.load(path_load_embeddings + 'test_labels_embedding.npy')

In [18]:
path_load_labels= '/home/app/src/embeddings/aa/'
train_labels = np.load(path_load_labels + 'train_labels_embedding.npy')
test_labels = np.load(path_load_labels + 'test_labels_embedding.npy')


Sequence model (RNN)

In [22]:
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), metrics=["accuracy"],
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5"
    filepath_log = '/home/app/src/experiments/inceptionv3_GRU/logs'

    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                              patience=3, min_lr=0.000001)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1)
    tensor_board = keras.callbacks.TensorBoard (filepath_log)

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint, reduce_lr, tensor_board],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()

2022-11-14 22:15:40.058485: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1195950080 exceeds 10% of free system memory.


Epoch 1/40
457/457 [==============================] - ETA: 0s - loss: 1.3998 - accuracy: 0.3506
Epoch 1: val_loss improved from inf to 1.20347, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5
457/457 [==============================] - 21s 27ms/step - loss: 1.3998 - accuracy: 0.3506 - val_loss: 1.2035 - val_accuracy: 0.6537 - lr: 1.0000e-04
Epoch 2/40
455/457 [============================>.] - ETA: 0s - loss: 1.1203 - accuracy: 0.5953
Epoch 2: val_loss improved from 1.20347 to 0.91868, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights_exp_005.h5
457/457 [==============================] - 9s 19ms/step - loss: 1.1205 - accuracy: 0.5952 - val_loss: 0.9187 - val_accuracy: 0.6625 - lr: 1.0000e-04
Epoch 3/40
457/457 [==============================] - ETA: 0s - loss: 0.9487 - accuracy: 0.6432
Epoch 3: val_loss improved from 0.91868 to 0.82405, saving model to /home/app/src/experiments/inceptionv3_GRU/exp_006/model.weights

In [ ]:
%tensorboard == logdir /home/app/src/experiments/inceptionv3_GRU/logs

UsageError: Line magic function `%tensorboard` not found.


Predictions

In [ ]:
# def prepare_single_video(frames):
#     frames = frames[None, ...]
#     frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
#     frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

#     for i, batch in enumerate(frames):
#         video_length = batch.shape[0]
#         length = min(MAX_SEQ_LENGTH, video_length)
#         for j in range(length):
#             frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
#         frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

#     return frame_features, frame_mask


# def sequence_prediction(path):
#     class_vocab = label_processor.get_vocabulary()

#     frames = load_video(os.path.join("test", path))
#     frame_features, frame_mask = prepare_single_video(frames)
#     probabilities = sequence_model.predict([frame_features, frame_mask])[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     return frames


# This utility is for visualization.
# Referenced from:
# https://www.tensorflow.org/hub/tutorials/action_recognition_with_tf_hub
# def to_gif(images):
#     converted_images = images.astype(np.uint8)
#     imageio.mimsave("animation.gif", converted_images, fps=10)
#     return embed.embed_file("animation.gif")


test_video = np.random.choice(test_df["video_name"].values.tolist())
print(f"Test video path: {test_video}")
test_frames = sequence_prediction(test_video)
# to_gif(test_frames[:MAX_SEQ_LENGTH])

Test video path: /workspaces/final-project-shot-type/data/shot-type-dataset/trailer_v3/test/shot_tt2517260_0047.mp4


NameError: name 'sequence_prediction' is not defined